<a href="https://colab.research.google.com/github/CalvHobbes/shop_india/blob/main/llamaindex/price%20comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4 requests

In [41]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
# prompt: check if in colab environment and if so, fetch helper functions for tools

import sys

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py


--2025-01-30 13:49:59--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1811 (1.8K) [text/plain]
Saving to: ‘search_Croma.py’

search_Croma.py     100%[===================>]   1.77K  --.-KB/s    in 0s      

2025-01-30 13:49:59 (24.5 MB/s) - ‘search_Croma.py’ saved [1811/1811]

--2025-01-30 13:49:59--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1460

In [ ]:
# Import the necessary functions
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance
import json

def test_functions():
  # Define the product query
  query = "iphone 16 pro max"

  # Fetch prices from Croma
  croma_prices = fetch_prices_from_croma(query)
  print("Croma Prices:")
  print(json.dumps(croma_prices, indent=4))

  # Fetch prices from Reliance Digital
  reliance_prices = fetch_prices_from_reliance(query)
  print("Reliance Digital Prices:")
  print(json.dumps(reliance_prices, indent=4))


# test_functions()





In [ ]:
!pip install llama-index llama-index-llms-openai

In [9]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAPI_KEY')

In [12]:

croma_tool = FunctionTool.from_defaults(
    fn=fetch_prices_from_croma,
    name="fetch_prices_from_croma",
    description="Fetches prices for a product from Croma.",
)

reliance_tool = FunctionTool.from_defaults(
    fn=fetch_prices_from_reliance,
    name="fetch_prices_from_reliance",
    description="Fetches prices for a product from Reliance Digital.",
)


In [19]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI

# model = OpenAI(model="gpt-3.5-turbo")
model = OpenAI(model="gpt-4o-mini")



In [28]:
instructions = '''
You are an intelligent assistant tasked with analyzing and combining product price data by executing one or more tools available to you. to run the tools you must provide them the name of the product the user is searching.. Your goal is to identify the most relevant product prices and present them in a structured format.  Execute all applicable tools.Follow these steps:

Combine Results: Analyze the product data from all websites. If product names or display names are slightly different but refer to the same product, use your judgment to group them together.

Identify the Best Prices: For each product, compare the prices across all websites and order them in ascending order (lowest price first).

Output Format: Present the results in the following structured format for each product:

Product Name: The most accurate or common name for the product.

For each store, include:

Store Display Name: The name of the store.

Product URL: A clickable link to the product page.

Price: The price of the product at that store.

Ensure the store-wise details are ordered by ascending price (cheapest first).

Handle Ambiguities: If product names or details are unclear, use context to determine if they refer to the same product. If uncertain, list them separately.

Sample Result:
Product Name: Wireless Noise-Cancelling Headphones
- Store: TechWorld, Product URL: [TechWorld Link], Price: $120
- Store: GadgetHub, Product URL: [GadgetHub Link], Price: $125
- Store: ElectroShop, Product URL: [ElectroShop Link], Price: $130

'''

In [33]:
# Use low level API - AgentWorker and AgentRunner
# Gives more control,can use non OpenAI model
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from typing import List, Tuple

class PriceCompAgentWorker(FunctionCallingAgentWorker):
       def __init__(self, llm, tools, **kwargs):
           super().__init__(llm=llm, tools=tools, **kwargs)
           self.system_prompt = kwargs.get("system_prompt")

       async def call_llm_with_tools(
           self,
           prompt: str,
           tool_strings: List[str],
           verbose: bool = False,
       ) -> Tuple[str, str]:
           # Prepend system prompt to the user prompt
           full_prompt = f"{self.system_prompt}\n\n{prompt}"

           # Call the LLM with the combined prompt and tool strings
           response, tool_code = await self.llm.apredict_and_call(
               full_prompt, tool_strings=tool_strings
           )
           return response, tool_code


agent_worker = PriceCompAgentWorker.from_tools(
    [croma_tool, reliance_tool],
    llm=model,
    system_prompt=instructions,
    verbose=True

)
agent_runner = AgentRunner(agent_worker)

In [34]:
from IPython.display import display, Markdown
chat_history = []
response = agent_runner.chat("What is the best price for LG 9kg  Front Load Washing Machine?", chat_history=chat_history)
display(Markdown(response.response))

Added user message to memory: What is the best price for LG 9kg  Front Load Washing Machine?
=== Calling Function ===
Calling function: fetch_prices_from_croma with args: {"product": "LG 9kg Front Load Washing Machine"}
=== Function Output ===
[{'name': 'LG 9kg 5 Star Fully Automatic Front Load Washing Machine (FHV1409Z4M.ABMQEIL, LG ThinQ with Wi-Fi, Middle Black)', 'url': 'https://www.croma.com/lg-9kg-5-star-fully-automatic-front-load-washing-machine-fhv1409z4m-abmqeil-lg-thinq-with-wi-fi-middle-black-/p/260606', 'price': 43990.0}, {'name': 'LG 9 kg 5 Star Wi-Fi Inverter Fully Automatic Front Load Washing Machine (FHB1209Z4B, Auto Restart, Platinum Black)', 'url': 'https://www.croma.com/lg-9-kg-5-star-wi-fi-inverter-fully-automatic-front-load-washing-machine-fhb1209z4b-auto-restart-platinum-black-/p/311332', 'price': 45990.0}, {'name': 'LG 9 kg 5 Star Inverter Fully Automatic Front Load Washing Machine (Vivace Pro, FHP1209Z5M.AMBQEIL, AI Direct Drive Technology, Middle Black)', 'url'

Here are the best prices for the LG 9kg Front Load Washing Machine:

### Product Name: LG 9kg Front Load Washing Machine

- **Store:** Reliance Digital  
  **Product URL:** [LG 9 Kg Fully Automatic Front Loading Washing Machine, FHB1209Z4B, Platinum Black](https://www.reliancedigital.in/lg-9-kg-fully-automatic-front-loading-washing-machine-fhb1209z4b-platinum-black/p/494460088)  
  **Price:** ₹39,990.00

- **Store:** Reliance Digital  
  **Product URL:** [LG 9 Kg Front Load Fully Automatic Washing Machine, FHV1409Z2M, Middle Black](https://www.reliancedigital.in/lg-9-kg-front-load-fully-automatic-washing-machine-fhv1409z2m-middle-black/p/493620608)  
  **Price:** ₹40,990.00

- **Store:** Croma  
  **Product URL:** [LG 9kg 5 Star Fully Automatic Front Load Washing Machine (FHV1409Z4M.ABMQEIL, LG ThinQ with Wi-Fi, Middle Black)](https://www.croma.com/lg-9kg-5-star-fully-automatic-front-load-washing-machine-fhv1409z4m-abmqeil-lg-thinq-with-wi-fi-middle-black-/p/260606)  
  **Price:** ₹43,990.00

- **Store:** Croma  
  **Product URL:** [LG 9 kg 5 Star Inverter Fully Automatic Front Load Washing Machine (FHP1209Z9B.ABLQEIL, In-built Heater, Steam Plus, Black)](https://www.croma.com/lg-9-kg-5-star-inverter-fully-automatic-front-load-washing-machine-fhp1209z9b-ablqeil-in-built-heater-steam-plus-black-/p/270905)  
  **Price:** ₹44,990.00

- **Store:** Croma  
  **Product URL:** [LG 9 kg 5 Star Wi-Fi Inverter Fully Automatic Front Load Washing Machine (FHB1209Z4B, Auto Restart, Platinum Black)](https://www.croma.com/lg-9-kg-5-star-wi-fi-inverter-fully-automatic-front-load-washing-machine-fhb1209z4b-auto-restart-platinum-black-/p/311332)  
  **Price:** ₹45,990.00

### Summary
The best price for the LG 9kg Front Load Washing Machine is ₹39,990.00 at Reliance Digital for the model FHB1209Z4B.

In [38]:
# Same thing using ReActAgent
from llama_index.core.agent import ReActAgent
from llama_index.core.agent.react import ReActChatFormatter

agent = ReActAgent.from_tools(
    [croma_tool, reliance_tool],
    llm=model,
    verbose=True,
    instructions=instructions,
    react_chat_formatter=ReActChatFormatter.from_defaults(
        context=instructions
    )
)


In [40]:
response = agent_runner.chat("What is the best price for LG 9kg  Front Load Washing Machine?", chat_history=chat_history)
display(Markdown(response.response))

Added user message to memory: What is the best price for LG 9kg  Front Load Washing Machine?
=== Calling Function ===
Calling function: fetch_prices_from_croma with args: {"product": "LG 9kg Front Load Washing Machine"}
=== Function Output ===
[{'name': 'LG 9kg 5 Star Fully Automatic Front Load Washing Machine (FHV1409Z4M.ABMQEIL, LG ThinQ with Wi-Fi, Middle Black)', 'url': 'https://www.croma.com/lg-9kg-5-star-fully-automatic-front-load-washing-machine-fhv1409z4m-abmqeil-lg-thinq-with-wi-fi-middle-black-/p/260606', 'price': 43990.0}, {'name': 'LG 9 kg 5 Star Wi-Fi Inverter Fully Automatic Front Load Washing Machine (FHB1209Z4B, Auto Restart, Platinum Black)', 'url': 'https://www.croma.com/lg-9-kg-5-star-wi-fi-inverter-fully-automatic-front-load-washing-machine-fhb1209z4b-auto-restart-platinum-black-/p/311332', 'price': 45990.0}, {'name': 'LG 9 kg 5 Star Inverter Fully Automatic Front Load Washing Machine (Vivace Pro, FHP1209Z5M.AMBQEIL, AI Direct Drive Technology, Middle Black)', 'url'

Here are the best prices for the LG 9kg Front Load Washing Machine:

### Product Name: LG 9kg Front Load Washing Machine

- **Store:** Reliance Digital  
  **Product URL:** [LG 9 Kg Fully Automatic Front Loading Washing Machine, FHB1209Z4B, Platinum Black](https://www.reliancedigital.in/lg-9-kg-fully-automatic-front-loading-washing-machine-fhb1209z4b-platinum-black/p/494460088)  
  **Price:** ₹39,990.00

- **Store:** Reliance Digital  
  **Product URL:** [LG 9 Kg Front Load Fully Automatic Washing Machine, FHV1409Z2M, Middle Black](https://www.reliancedigital.in/lg-9-kg-front-load-fully-automatic-washing-machine-fhv1409z2m-middle-black/p/493620608)  
  **Price:** ₹40,990.00

- **Store:** Croma  
  **Product URL:** [LG 9kg 5 Star Fully Automatic Front Load Washing Machine (FHV1409Z4M.ABMQEIL, LG ThinQ with Wi-Fi, Middle Black)](https://www.croma.com/lg-9kg-5-star-fully-automatic-front-load-washing-machine-fhv1409z4m-abmqeil-lg-thinq-with-wi-fi-middle-black-/p/260606)  
  **Price:** ₹43,990.00

- **Store:** Croma  
  **Product URL:** [LG 9 Kg 5 Star Inverter Fully Automatic Front Load Washing Machine (FHP1209Z9B.ABLQEIL, In-built Heater, Steam Plus, Black)](https://www.croma.com/lg-9-kg-5-star-inverter-fully-automatic-front-load-washing-machine-fhp1209z9b-ablqeil-in-built-heater-steam-plus-black-/p/270905)  
  **Price:** ₹44,990.00

- **Store:** Croma  
  **Product URL:** [LG 9 kg 5 Star Wi-Fi Inverter Fully Automatic Front Load Washing Machine (FHB1209Z4B, Auto Restart, Platinum Black)](https://www.croma.com/lg-9-kg-5-star-wi-fi-inverter-fully-automatic-front-load-washing-machine-fhb1209z4b-auto-restart-platinum-black-/p/311332)  
  **Price:** ₹45,990.00

- **Store:** Croma  
  **Product URL:** [LG 9/5 kg 5 Star Fully Automatic Front Load Washer Dryer(FHD0905STB.ABLQEIL, In-built Heater, Black)](https://www.croma.com/lg-9-5-kg-5-star-fully-automatic-front-load-washer-dryer-fhd0905stb-ablqeil-in-built-heater-black-/p/274043)  
  **Price:** ₹55,490.00

This structured format provides a clear comparison of the best prices available for the LG 9kg Front Load Washing Machine across different stores.

In [20]:
from llama_index.agent.openai import OpenAIAgent


# Initialize the OpenAIAgent with the tools and model - OpenAIAgent is a higher level of abstraction as compared
# to using FunctionCallingAgentWorker with AgentRunner
agent = OpenAIAgent.from_tools(
    [croma_tool, reliance_tool],
    llm=model,
    verbose=True,
    instructions=instructions
)


In [27]:
from IPython.display import display, Markdown

chat_history = []
response = agent.chat("What is the best price for LG 9kg  Front Load Washing Machine?", chat_history=chat_history)
display(Markdown(response.response))

Added user message to memory: What is the best price for LG 9kg  Front Load Washing Machine?
=== Calling Function ===
Calling function: fetch_prices_from_croma with args: {"product": "LG 9kg Front Load Washing Machine"}
Got output: [{'name': 'LG 9kg 5 Star Fully Automatic Front Load Washing Machine (FHV1409Z4M.ABMQEIL, LG ThinQ with Wi-Fi, Middle Black)', 'url': 'https://www.croma.com/lg-9kg-5-star-fully-automatic-front-load-washing-machine-fhv1409z4m-abmqeil-lg-thinq-with-wi-fi-middle-black-/p/260606', 'price': 43990.0}, {'name': 'LG 9 kg 5 Star Wi-Fi Inverter Fully Automatic Front Load Washing Machine (FHB1209Z4B, Auto Restart, Platinum Black)', 'url': 'https://www.croma.com/lg-9-kg-5-star-wi-fi-inverter-fully-automatic-front-load-washing-machine-fhb1209z4b-auto-restart-platinum-black-/p/311332', 'price': 45990.0}, {'name': 'LG 9 kg 5 Star Inverter Fully Automatic Front Load Washing Machine (Vivace Pro, FHP1209Z5M.AMBQEIL, AI Direct Drive Technology, Middle Black)', 'url': 'https://w

Here are the best prices for the LG 9kg Front Load Washing Machine from Croma and Reliance Digital:

### Croma
1. **[LG 9kg 5 Star Fully Automatic Front Load Washing Machine (FHV1409Z4M.ABMQEIL, LG ThinQ with Wi-Fi, Middle Black)](https://www.croma.com/lg-9kg-5-star-fully-automatic-front-load-washing-machine-fhv1409z4m-abmqeil-lg-thinq-with-wi-fi-middle-black-/p/260606)** - ₹43,990.00
2. **[LG 9 kg 5 Star Wi-Fi Inverter Fully Automatic Front Load Washing Machine (FHB1209Z4B, Auto Restart, Platinum Black)](https://www.croma.com/lg-9-kg-5-star-wi-fi-inverter-fully-automatic-front-load-washing-machine-fhb1209z4b-auto-restart-platinum-black-/p/311332)** - ₹45,990.00
3. **[LG 9 Kg 5 Star Inverter Fully Automatic Front Load Washing Machine (FHP1209Z9B.ABLQEIL, In-built Heater, Steam Plus, Black)](https://www.croma.com/lg-9-kg-5-star-inverter-fully-automatic-front-load-washing-machine-fhp1209z9b-ablqeil-in-built-heater-steam-plus-black-/p/270905)** - ₹44,990.00

### Reliance Digital
1. **[LG 9 Kg Fully Automatic Front Loading Washing Machine (FHB1209Z4B, Platinum Black)](https://www.reliancedigital.in/lg-9-kg-fully-automatic-front-loading-washing-machine-fhb1209z4b-platinum-black/p/494460088)** - ₹39,990.00
2. **[LG 9 Kg Front Load Fully Automatic Washing Machine (FHV1409Z2M, Middle Black)](https://www.reliancedigital.in/lg-9-kg-front-load-fully-automatic-washing-machine-fhv1409z2m-middle-black/p/493620608)** - ₹40,990.00

### Best Price
The best price for the LG 9kg Front Load Washing Machine is **₹39,990.00** at Reliance Digital for the model FHB1209Z4B.